In [25]:
! pip install -r requirements.txt --quiet

# Creating and Deploying an Azure AI Agent Using SDK and Multiple Tools

This notebook demonstrates how to create and deploy an Azure AI Agent using the Azure SDK, with tools generated from an OpenAPI endpoint and Azure AI Search. It provides a coded example that mirrors the portal example demonstrated in this workshop, offering a programmatic approach to deploying agents with OpenAPI-driven tools.


🔗 [How to use Azure AI Agent Service with OpenAPI Specified Tools](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/openapi-spec?tabs=python&pivots=code-example)

🔗 [Use an existing AI Search index with the Azure AI Search tool](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-ai-search?tabs=azurecli%2Cpython&pivots=code-examples)

In [1]:

from os import environ
from dotenv import load_dotenv
import jsonref
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails,AzureAISearchTool,AzureAISearchQueryType,ToolSet
import requests

load_dotenv(override=True)

True

## Create a project client

In [2]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=environ["AZURE_AI_PROJECT_CONNECTION_STRING"],
)

## Create the OpenAPI Spec tool definition

Our Agent will create its tools from our deployed OpenAPI endpoints, for this to work we will need to pass in the openapi.json spec document.

In [3]:
spec_url = environ['OPENAPI_URL']
response = requests.get(spec_url)
response.raise_for_status()


openapi_spec = response.json()

auth = OpenApiAnonymousAuthDetails()

openapi = OpenApiTool(name="get_energy_usage", spec=openapi_spec, description="Retrieve data center energy, power, or consumption topic", auth=auth)


## Create the Azure AI Search tool definition

In [4]:
project_client.connections.get(connection_name='hub-foundry-lab-s48waambmtn8-connection-AzureAISearch')

In [6]:
# Extract the connection list.
conn_list = project_client.connections._list_connections()["value"]
conn_id = ""

# Search in the metadata field of each connection in the list for the azure_ai_search type and get the id value to establish the variable
for conn in conn_list:
    metadata = conn["properties"].get("metadata", {})
    print(metadata)
    if 'searchServices' in metadata.get("ResourceId", ""):
        conn_id = conn["id"]
        break

ai_search = AzureAISearchTool( index_connection_id=conn_id, index_name=environ['AZURE_AI_SEARCH_INDEX'],query_type= AzureAISearchQueryType.VECTOR_SIMPLE_HYBRID)

{'ApiType': 'Azure', 'ResourceId': '/subscriptions/71dcf7f8-6dda-4243-a84c-88833a4d8278/resourceGroups/rg-foundry-lab-eastus2-s48waambmtn8/providers/Microsoft.Search/searchServices/srch-foundry-lab-s48waambmtn8', 'ApiVersion': '2024-05-01-preview', 'DeploymentApiVersion': '2023-11-01'}


## Create an agent and a thread

In [7]:
instructions =  """
You are a helpful and knowledgeable assistant that helps users understand and monitor data center energy usage.

Use the provided tools to retrieve accurate, real-time, and historical data on energy consumption, efficiency metrics (such as PUE), carbon footprint, and resource utilization across different data center regions or racks. These tools include:
- An energy data tool that provides structured metrics and time-series data.
- A document lookup tool that retrieves relevant information from a vector store of technical documents and operational reports.

Always respond with concise and clear summaries, including units of measurement (e.g., kWh, %, kg CO₂e). If applicable, compare usage across regions or time ranges to highlight trends or inefficiencies.

Only respond if the requested information is found using the tools. If the user asks something unrelated to data center energy or the answer cannot be retrieved from the available tools, respond with:
"I'm unable to find that information based on the current data."
"""

toolset = ToolSet()
toolset.add(openapi)
toolset.add(ai_search)

agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="Data Center Agent",
    instructions=instructions,
    toolset=toolset
)


print(f"Created agent, ID: {agent.id}")

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")



Created agent, ID: asst_2xc4Pmgo0aUsKCrASXYRFLqh
Created thread, ID: thread_aUXwg3P7ovz7LpLiEF6Tsp2t


## Create a run and check the output

In [8]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="What data centers have the highest co2 emissions?",
)
print(f"Created message, ID: {message.id}")

# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Created message, ID: msg_IGGLHAnHI7ePMD0yv2iXhVom
Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_eCMLl3BxzSYLx7MfgzwFTqXR', 'object': 'thread.message', 'created_at': 1746140996, 'assistant_id': 'asst_2xc4Pmgo0aUsKCrASXYRFLqh', 'thread_id': 'thread_aUXwg3P7ovz7LpLiEF6Tsp2t', 'run_id': 'run_WrOhThVtc6672TqkB4fIvykS', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The data center currently with the highest CO₂ emissions is **DC-CHI2** (Zone C3), which has recorded emissions of **153.31 kg CO₂e**. It is followed by **DC-NYC1** (Zone B2) emitting **118.07 kg CO₂e**. These centers are likely experiencing increased power draw, non-renewable energy usage, or operational inefficiencies affecting emissions.', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_IGGLHAnHI7ePMD0yv2iXhVom', 'object': 'thread.message', 'created_at': 1746140991, 'assistant_id': None, 'thread_id': 'thread_aUXwg3P7ovz7LpLiEF6Tsp2t',

## Delete the assistant when done

Comment the code below to save the agent to your Azure AI Froundry project.


In [ ]:

project_client.agents.delete_agent(agent.id)
print("Deleted agent")